## Context-Enriched Retrieval in RAG

### Context-Enriched RAG의 단계

**1. 데이터 수집:**
다양한 외부 자료(예: PDF, 웹 문서 등)에서 텍스트를 추출한다.
문단 또는 페이지 단위로 원시 데이터를 구성한다.

**2. 컨텍스트가 겹치는 청크 분할:**
텍스트를 일정 크기의 청크로 분할하되, 각 청크가 일정 범위만큼 이전/다음 문장과 겹치도록 한다.
예: 512 토큰 청크를 128 토큰씩 겹치게(sliding window) 구성.
이로 인해 각 청크는 더 풍부한 문맥을 포함하게 된다.

**3. 임베딩 생성:**
청크를 임베딩 모델(BGE, OpenAI, Cohere 등)을 사용해 숫자 벡터로 변환한다.
이렇게 생성된 벡터는 벡터 검색 엔진(예: FAISS, Weaviate 등)에 저장된다.

**4. 문맥 인식 검색 (Context-Aware Retrieval):**
사용자의 질의에 대해 가장 관련 있는 청크를 검색한 후, 해당 청크의 앞뒤 인접 청크도 함께 가져온다.
이 방식은 중요한 맥락이나 배경 지식을 보완하여, 언어 모델이 더 완전한 이해를 바탕으로 응답을 생성할 수 있게 한다.

**5. 응답 생성:**
검색된 메인 청크와 그 주변 문맥을 LLM에 입력하여 응답을 생성한다.
응답은 보다 자연스럽고 사실에 근거한 결과를 제공한다.

**6. 응답 평가:**
생성된 응답의 정확도, 일관성, 관련성을 기준으로 평가한다.
정량 지표(예: ROUGE, BLEU) 또는 정성 평가(GPT-4 평가, 사용자 피드백 등)를 활용할 수 있다.


### Context-Enriched RAG의 장점

- 문맥 유지: 질문과 관련된 핵심 내용뿐 아니라 그 주변 정보를 함께 제공함으로써 응답의 논리성과 완성도를 높인다.
- 부분 정보로 인한 오류 감소: 중요한 배경이나 정의가 누락되지 않도록 하여, hallucination(환각 현상)을 줄일 수 있다.
- 검색 신뢰도 향상: 검색 결과가 고립되지 않으므로, LLM이 보다 풍부한 정보를 기반으로 reasoning할 수 있다.

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출하고 전체 텍스트를 반환합니다.

    Args:
        pdf_path (str): PDF 파일 경로.

    Returns:
        str: 추출된 전체 텍스트.
    """
    # PDF 파일을 엽니다.
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 문자열 초기화

    # PDF의 각 페이지를 순회하며 텍스트를 추출합니다.
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 객체를 가져옵니다.
        text = page.get_text("text")  # 해당 페이지에서 텍스트를 추출합니다.
        all_text += text  # 추출된 텍스트를 누적합니다.

    # 전체 텍스트를 반환합니다.
    return all_text

## Chunking the Extracted Text

In [4]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로 겹치게 분할합니다.

    Args:
        text (str): 분할할 텍스트.
        n (int): 각 청크의 문자 수.
        overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
        List[str]: 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap)만큼 이동하면서 텍스트를 청크로 나눕니다.
    for i in range(0, len(text), n - overlap):
        # i부터 i+n까지의 텍스트를 청크로 잘라서 추가합니다.
        chunks.append(text[i:i + n])

    # 생성된 청크 리스트를 반환합니다.
    return chunks

## Extracting and Chunking Text from a PDF File

In [5]:
# PDF 파일 경로를 정의합니다.
pdf_path = "../../dataset/AI_Understanding.pdf"

# PDF 파일에서 텍스트를 추출합니다.
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트를 1000자 단위, 200자 중첩으로 청크 분할합니다.
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 텍스트 청크의 개수를 출력합니다.
print("청크 개수:", len(text_chunks))

# 첫 번째 텍스트 청크를 출력합니다.
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

청크 개수: 21

첫 번째 텍스트 청크:
인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다. 이 용어는 추론, 의미 발견, 일반화, 과거 
경험으로부터의 학습 능력 등 인간의 특징적인 지적 프로세스가 부여된 시스템을 
개발하는 프로젝트에 자주 적용됩니다. 지난 수십 년 동안 컴퓨팅 성능과 데이터 가용성의 
발전으로 AI의 개발과 배포가 크게 가속화되었습니다. 
역사적 맥락 
인공 지능에 대한 개념은 수세기 동안 존재해 왔으며 종종 신화와 소설에 묘사되기도 
했습니다. 하지만 공식적인 AI 연구 분야는 20세기 중반에 시작되었습니다. 1956년 
다트머스 워크숍은 AI의 발상지로 널리 알려져 있습니다. 초기 AI 연구는 문제 해결과 
상징적 방법에 중점을 두었습니다. 1980년대에는 전문가 시스템이 등장했고, 1990년대와 
2000년대에는 머신러닝과 신경망이 발전했습니다. 최근 딥러닝의 획기적인 발전은 이 
분야에 혁신을 가져왔습니다. 
현대 관측 
최신 AI 시스템은 일상 생활에서 점점 더 널리 보급되고 있습니다. Siri와 Alexa 같은 가상 
비서부터 스트리밍 서비스 및 소셜 미디어의 추천 알고리즘에 이르기까지 AI는 우리의 
생활, 업무, 상호 작용 방식에 영향을 미치고 있습니다. 자율 주행 자동차, 첨단 의료 진단, 
정교한 재무 모델링 도구의 개발은 AI가 광범위하고 성장하는 응용 분야를 보여줍니다. 
윤리적 영향, 편견, 일자리 대체에 대한 우려도 점점 더 커지고 있습니다. 
 
2장: 인공 지능의 핵심 개념 
머신 러닝 
머신러닝(ML)은 명시적으로 프로그래밍하지 않고도 시스템이 데이터로부터 학습할 수 
있도록 하는 데 중점을 둔 AI의 하위 집합입니다. ML 알고리즘은 더 많은 데이터에 
노출됨에 따라 패턴을 식별하고 예측하며 시간이 지남에 따라 성능을 개선합니다. 
지도 학습 
지도 학습에서는 입력 데이터가

## Creating Embeddings for Text Chunks

In [6]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
        text (str): 임베딩을 생성할 입력 텍스트 또는 텍스트 리스트.
        model (str): 사용할 임베딩 모델. 기본값은 "BAAI/bge-en-icl".

    Returns:
        dict: OpenAI API의 응답 객체 (임베딩 포함).
    """
    # 지정된 모델을 사용하여 텍스트에 대한 임베딩을 생성합니다.
    response = client.embeddings.create(
        model=model,
        input=text
    )

    # 생성된 임베딩 응답을 반환합니다.
    return response

# 텍스트 청크에 대해 임베딩을 생성합니다.
response = create_embeddings(text_chunks)

## Implementing Context-Aware Semantic Search

In [7]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
        vec1 (np.ndarray): 첫 번째 벡터.
        vec2 (np.ndarray): 두 번째 벡터.

    Returns:
        float: 두 벡터 간의 코사인 유사도.
    """
    # 두 벡터의 내적을 계산하고, 각 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [8]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    가장 관련성 높은 청크와 그 주변 청크들을 함께 검색합니다.

    Args:
        query (str): 검색 쿼리.
        text_chunks (List[str]): 텍스트 청크 리스트.
        embeddings (List[dict]): 텍스트 청크에 대한 임베딩 리스트.
        k (int): 검색할 관련 청크 개수 (현재는 상위 1개만 사용).
        context_size (int): 관련 청크 주변에 포함할 청크 수.

    Returns:
        List[str]: 관련성 높은 청크 및 문맥 정보를 포함한 텍스트 청크 리스트.
    """
    # 쿼리를 임베딩 벡터로 변환합니다.
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []

    # 각 청크 임베딩과 쿼리 임베딩 간의 유사도 점수를 계산합니다.
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(
            np.array(query_embedding),
            np.array(chunk_embedding.embedding)
        )
        similarity_scores.append((i, similarity_score))  # (인덱스, 유사도) 저장

    # 유사도 점수를 기준으로 내림차순 정렬합니다.
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 가장 관련성 높은 청크의 인덱스를 가져옵니다.
    top_index = similarity_scores[0][0]

    # 문맥 포함 범위를 정의합니다.
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # 해당 범위의 청크들을 반환합니다.
    return [text_chunks[i] for i in range(start, end)]

## Running a Query with Context Retrieval

In [9]:
# JSON 파일에서 검증 데이터를 불러옵니다.
with open('../../dataset/validation.json') as f:
    data = json.load(f)

# 첫 번째 질문을 쿼리로 추출합니다.
query = data[0]['question']

# 문맥 확장 검색을 통해 관련 청크와 그 주변 청크를 검색합니다.
# 매개변수 설명:
# - query: 검색할 질문
# - text_chunks: PDF에서 분할한 텍스트 청크
# - response.data: 텍스트 청크들의 임베딩
# - k=1: 가장 유사한 청크 하나만 선택
# - context_size=1: 해당 청크 앞뒤로 1개씩 문맥 포함
top_chunks = context_enriched_search(query, text_chunks, response.data, k=1, context_size=1)

# 쿼리를 출력합니다.
print("쿼리:", query)

# 검색된 각 청크를 번호와 구분선을 포함하여 출력합니다.
for i, chunk in enumerate(top_chunks):
    print(f"컨텍스트 {i + 1}:\n{chunk}\n{'-' * 30}")

쿼리: '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?
컨텍스트 1:
 구호 활동을 지원함으로써 재난 대응을 
강화합니다. AI 기반 시스템은 상황 인식을 개선하고, 리소스 할당을 최적화하며, 대응자 
간의 조율을 강화합니다. 
 
18장: AI에서 정부와 정책의 역할 
AI 전략 및 정책 프레임워크 
전 세계 각국 정부는 개발과 배포를 위해 AI 전략과 정책 프레임워크를 개발하고 있습니다. 
이러한 프레임워크는 윤리적 고려 사항을 다루고, 혁신을 촉진하며, 책임감 있는 AI 관행을 
보장합니다. 
AI 규제 
AI 규제는 복잡하고 진화하는 분야입니다. 각국 정부는 편향성, 투명성, 개인정보 보호, 안전 
등의 문제를 해결하기 위한 규제를 고려하고 있습니다. 혁신과 윤리적 고려 사항의 균형을 
맞추는 것이 핵심 과제입니다. 
AI 연구 및 개발을 위한 자금 지원 
정부는 AI 연구 및 개발에 자금을 지원하는 데 중요한 역할을 합니다. 공공 자금은 기초 
연구, 응용 연구, AI 인프라 개발을 지원합니다. 정부의 투자는 혁신을 촉진하고 협업을 
촉진합니다. 
국제 협력 
AI가 제시하는 글로벌 도전과 기회를 해결하기 위해서는 국제 협력이 필수적입니다. 
여기에는 지식 공유, 표준 개발, 국경을 넘어 책임감 있는 AI 관행을 장려하는 것이 
포함됩니다. 
대중 참여 및 교육 
AI에 대한 토론에 대중을 참여시키는 것은 신뢰를 구축하고 AI 개발이 사회적 가치에 
부합하도록 보장하는 데 매우 중요합니다. 교육 및 인식 제고 캠페인은 대중에게 AI와 그 
영향력, 잠재력에 대해 알립니다. 
 
19장: AI와 윤리 
윤리적 AI의 원칙 
윤리적 AI 원칙은 공정하고 투명하며 책임감 있고 사회에 유익한 AI 시스템의 개발과 
배포를 보장하기 위한 지침입니다. 주요 원칙에는 인권 존중, 개인정보 보호, 비차별, 
공익성이 포함됩니다. 
AI의 편향성 해결 
AI 시스템은 데이터에 존재하는 편견을 유전하고 증폭시켜 불공정하거나 차별적인 결과를 

## Generating a Response Using Retrieved Context

In [10]:
# AI 어시스턴트를 위한 시스템 프롬프트를 정의합니다.
system_prompt = (
    "당신은 주어진 컨텍스트에 기반하여 엄격하게 대답하는 AI 어시스턴트입니다. "
    "제공된 컨텍스트에서 직접적으로 답변을 도출할 수 없는 경우, 다음과 같이 응답하십시오: "
    "'I do not have enough information to answer that.'"
)

def generate_response(system_prompt, user_message, model="gpt-4o-mini"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델의 응답을 생성합니다.

    Args:
        system_prompt (str): AI의 동작 방식을 안내하는 시스템 프롬프트.
        user_message (str): 사용자 쿼리 또는 메시지.
        model (str): 사용할 AI 모델. 

    Returns:
        dict: AI 모델의 응답 객체.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 검색된 상위 청크들을 기반으로 사용자 프롬프트를 생성합니다.
user_prompt = "\n".join([
    f"Context {i + 1}:\n{chunk}\n---------------------------------\n"
    for i, chunk in enumerate(top_chunks)
])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# AI 응답을 생성합니다.
ai_response = generate_response(system_prompt, user_prompt)

## Evaluating the AI Response

In [11]:
# 평가 시스템을 위한 시스템 프롬프트를 정의합니다.
evaluate_system_prompt = (
    "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. "
    "AI 어시스턴트의 응답이 정답과 매우 유사하면 점수 1을 부여하세요. "
    "응답이 정답과 맞지 않거나 부적절하다면 점수 0을 부여하세요. "
    "부분적으로 일치하거나 애매한 경우에는 점수 0.5를 부여하세요."
)

# 사용자 쿼리, AI 응답, 정답, 평가 지침을 포함한 평가용 프롬프트를 생성합니다.
evaluation_prompt = (
    f"User Query: {query}\n"
    f"AI Response:\n{ai_response.choices[0].message.content}\n"
    f"True Response: {data[0]['ideal_answer']}\n"
    f"{evaluate_system_prompt}"
)

# 평가 시스템을 사용하여 점수를 생성합니다.
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 결과를 출력합니다.
print(evaluation_response.choices[0].message.content)

점수 0을 부여합니다. AI 어시스턴트의 응답은 질문에 대한 정보가 전혀 없으며, 정답과 관련이 없습니다.
